In [57]:
import sys,os
sys.path.append(os.pardir)
import reversi.reversi as rev
import numpy as np
import torch
import torch.nn as nn

import importlib

# モジュールの変更を反映
importlib.reload(rev)

<module 'reversi.reversi' from '/Users/haruni20/Desktop/BoardGames/Othello/alphazero/../reversi/reversi.py'>

In [58]:
board = rev.Board()

In [59]:
print(board)

[[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0 -1  1  0  0  0]
 [ 0  0  0  1 -1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]


In [60]:
board.push(np.random.choice(board.legal_moves))

In [61]:
%%timeit
for i in range(5):
    board.reset()
    while not board.end:
        board.push(np.random.choice(board.legal_moves))
    print(board)

[[ 1  1  1  1  1  1  1  1]
 [-1 -1  1 -1 -1 -1  1  1]
 [-1  1  1 -1 -1  1 -1  1]
 [-1 -1  1 -1 -1 -1  1  1]
 [-1 -1  1 -1 -1  1  1  1]
 [-1 -1  1 -1  1 -1  1 -1]
 [-1 -1  1  1 -1 -1 -1 -1]
 [-1 -1  1 -1 -1 -1 -1 -1]]
[[ 1  1 -1 -1 -1 -1 -1 -1]
 [ 1  1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [ 1 -1 -1  1 -1 -1 -1 -1]
 [ 1  1 -1  1 -1 -1 -1 -1]
 [ 1  1  1 -1 -1 -1 -1 -1]
 [ 1  1  1  1 -1 -1 -1 -1]
 [ 1  1  1  1  1  1  1  1]]
[[ 1  1 -1 -1  1  1  1  1]
 [ 1  1 -1 -1  1  1  1  1]
 [ 1  1  1  1  1 -1 -1 -1]
 [ 1  1  1  1  1  1 -1 -1]
 [ 1 -1  1  1  1 -1  1 -1]
 [ 1 -1  1 -1 -1  1  1 -1]
 [ 1  1 -1  1  1 -1  1 -1]
 [ 1  1  1  1  1 -1 -1 -1]]
[[-1 -1 -1 -1  1  1 -1 -1]
 [-1 -1  1  1  1  1  1  1]
 [-1  1  1 -1  1  1  1  1]
 [-1  1  1 -1 -1  1  1  1]
 [-1  1 -1 -1 -1  1  1  1]
 [-1  1 -1 -1  1 -1  1  1]
 [-1 -1 -1 -1 -1  1  1  1]
 [-1 -1 -1 -1 -1 -1 -1 -1]]
[[ 1 -1  1  1  1  1  1  1]
 [ 1 -1 -1  1  1 -1  1 -1]
 [ 1 -1  1 -1 -1  1 -1 -1]
 [ 1 -1 -1  1  1 -1 -1 -1]
 [ 1 -1 -1  1  1 -1  1 -